# Motion Correction Tutorial

This Jupyter Notebook presents an example of the application of motion correction in T1 and DWI MRI images.


The motion correction algorithm uses Model-Driven Registration (MDR). `ukat` calls the python package `mdreg` and its source code can be found at: https://github.com/QIB-Sheffield/mdreg


Start by importing the required libraries and defining some settings:

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from ukat.data import fetch
from ukat.utils.tools import convert_to_pi_range
from ukat.mapping.t1 import T1, magnitude_correct
from ukat.mapping.diffusion import ADC, DTI
from ukat.moco.mdr import MotionCorrection

# Ensure figures are rendered in the notebook
%matplotlib inline

# Initialise output path for the Model-Driven Registration process
directory = 'motion_correction_output'
os.makedirs(directory, exist_ok=True)
OUTPUT_DIR = os.path.join(os.getcwd(), directory)

Fetch UKRIN-MAPS T1 test data

In [ ]:
magnitude, phase, affine_t1, ti, tss = fetch.t1_philips(2)
# Prepare the T1 modelling input arguments
phase = convert_to_pi_range(phase)
complex_data = magnitude * (np.cos(phase) + 1j * np.sin(phase)) # convert magnitude and phase into complex data
ti = np.array(ti) * 1000  # convert TIs to ms
magnitude_corrected = np.squeeze(magnitude_correct(complex_data))
# Pre-processing as preparation for the Model-Driven Registration process
list_input_parameters_t1 = [affine_t1, ti, 0, None, None, 3, False]

In case you wish to test this with a different T1 dataset, see below a code snippet where a NIfTI file is read in Python.

```python
import nibabel as nib
image_file = os.path.join("C:\\{Users}\\{user}\\Desktop\\{folder}", "{file_name}.nii.gz")
magnitude_corrected = nib.load(image_file).get_fdata()
affine_t1 = nib.load(image_file).affine
ti = np.arange(0.1, 2.001, 0.1) * 1000  # convert TIs to ms
# Pre-processing as preparation for the Model-Driven Registration process
list_input_parameters_t1 = [affine_t1, ti, 0, None, None, 3, False]
```

Fetch UKRIN-MAPS DWI test data

In [ ]:
pixel_array, affine_dwi, bvals, bvecs = fetch.dwi_philips()
# Pre-processing as preparation for the Model-Driven Registration process
list_input_parameters_dwi = [affine_dwi, bvals, None, False]

In case you wish to test this with a different DWI dataset, see below a code snippet where a NIfTI file is read in Python.

```python
import nibabel as nib
from dipy.io import read_bvals_bvecs
image_file = os.path.join("C:\\{Users}\\{user}\\Desktop\\{folder}", "{file_name}.nii.gz")
bvals_file = os.path.join("C:\\{Users}\\{user}\\Desktop\\{folder}", "{file_name}.bval")
bvecs_file = os.path.join("C:\\{Users}\\{user}\\Desktop\\{folder}", "{file_name}.bvec")
pixel_array = nib.load(image_file).get_fdata()
affine_dwi = nib.load(image_file).affine
bvals, bvecs = read_bvals_bvecs(bvals_file, bvecs_file)
# Pre-processing as preparation for the Model-Driven Registration process
list_input_parameters_dwi = [affine_dwi, bvals, None, False]
```

Set bounding box masks for motion correction

In [ ]:
mask_flag = True
# The bounding box is set to 1/6 of the Field Of View (FOV). This ratio can be changed in the lines below.
if mask_flag == True:
    x_t1 = np.shape(magnitude_corrected)[0]
    y_t1 = np.shape(magnitude_corrected)[1]
    x_dwi = np.shape(pixel_array)[0]
    y_dwi = np.shape(pixel_array)[1]
    mask_moco_t1 = np.ones(np.shape(magnitude_corrected))
    mask_moco_dwi = np.ones(np.shape(pixel_array))
    mask_moco_t1[:int(x_t1/6), ...] = 0
    mask_moco_t1[int(5*x_t1/6):, ...] = 0
    mask_moco_t1[:, :int(y_t1/6), ...] = 0
    mask_moco_t1[:, int(5*y_t1/6):, ...] = 0
    mask_moco_dwi[:int(x_dwi/6), ...] = 0
    mask_moco_dwi[int(5*x_dwi/6):, ...] = 0
    mask_moco_dwi[:, :int(y_dwi/6), ...] = 0
    mask_moco_dwi[:, int(5*y_dwi/6):, ...] = 0
else:
    mask_moco_t1 = None
    mask_moco_dwi = None

## T1 Model-Driven Registration

In [ ]:
t1_registration = MotionCorrection(magnitude_corrected, affine_t1, 'T1_Moco', list_input_parameters_t1, convergence=1, multithread=False, log=False, mask=mask_moco_t1)

Extract and save the motion corrected T1 results.

In [ ]:
# Get the T1 coregistered pixel_array and the array with the difference between the original and the motion corrected
t1_moco_array = t1_registration.get_coregistered()
t1_moco_diff_array = t1_registration.get_diff_orig_coreg()
t1_registration.get_elastix_parameters(output_directory=OUTPUT_DIR, base_file_name='T1_Elastix', export=True)
t1_registration.get_improvements(output_directory=OUTPUT_DIR, base_file_name='T1_improvements', export=True)
# Save motion corrected T1 sequence to NIfTI
t1_registration.to_nifti(output_directory=OUTPUT_DIR, base_file_name='T1_motion_corrected', maps=['original', 'coregistered', 'difference', 'model_fit', 'mask', 'deformation_field', 'parameters'])
t1_registration.to_gif(output_directory=OUTPUT_DIR, base_file_name='T1_motion_corrected', maps=['original', 'coregistered', 'difference', 'model_fit', 'deformation_field'])

Perform T1 map calculation in the original and co-registered T1 images.

In [ ]:
# Calculate maps from the original T1 sequence save as niftis
t1_mapper = T1(magnitude_corrected, ti, affine=affine_t1, multithread=False, parameters=3)
t1_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='T1_original', maps='all')

# Calculate maps from the motion corrected T1 sequence and save as niftis
t1_moco_mapper = T1(t1_moco_array, ti, affine=affine_t1, multithread=False, parameters=3)
t1_moco_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='T1_moco', maps='all')

Display the difference between the original T1 images and the co-registered images.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

# Display the T1 original data
im = ax1.imshow(np.rot90(magnitude_corrected[..., 2, 10]), clim=(0, 1000))
cb = fig.colorbar(im, ax=ax1)
cb.set_label('T1 Original')
ax1.axis('off')

# Display the T1 co-registered data
im2 = ax2.imshow(np.rot90(t1_moco_array[..., 2, 10]), clim=(0, 1000))
cb = fig.colorbar(im2, ax=ax2)
cb.set_label('T1 Moco')
ax2.axis('off')

# Display the difference image between T1 original data and T1 co-registered data
im3 = ax3.imshow(np.rot90(t1_moco_diff_array[..., 2, 10]), clim=(0, 1000))
cb = fig.colorbar(im3, ax=ax3)
cb.set_label('T1 Moco Difference')
ax3.axis('off')

plt.show()

Plot the T1 maps from the original and co-registered images.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))

# Display the T1 map generated from the original data
im = ax1.imshow(np.rot90(t1_mapper.t1_map[..., 2]), cmap='inferno', clim=(250, 2250))
cb = fig.colorbar(im, ax=ax1)
cb.set_label('$T_1 Original$ (ms)')
ax1.axis('off')

# Display the T1 map generated from the motion corrected data
im2 = ax2.imshow(np.rot90(t1_moco_mapper.t1_map[..., 2]), cmap='inferno', clim=(250, 2250))
cb = fig.colorbar(im2, ax=ax2)
cb.set_label('$T_1 Moco$ (ms)')
ax2.axis('off')

plt.show()

## DWI Model-driven registration

In [ ]:
dwi_registration = MotionCorrection(pixel_array, affine_dwi, 'DWI_Moco', list_input_parameters_dwi, convergence=1, multithread=False, log=False, mask=mask_moco_dwi)

Extract and save the motion corrected DWI results.

In [ ]:
# Get the DWI coregistered pixel_array and the array with the difference between the original and the motion corrected
dwi_moco_array = dwi_registration.get_coregistered()
dwi_moco_diff_array = dwi_registration.get_diff_orig_coreg()
dwi_registration.get_elastix_parameters(output_directory=OUTPUT_DIR, base_file_name='DWI_Elastix', export=True)
dwi_registration.get_improvements(output_directory=OUTPUT_DIR, base_file_name='DWI_improvements', export=True)
# Save motion corrected diffusion sequence to NIfTI
dwi_registration.to_nifti(output_directory=OUTPUT_DIR, base_file_name='DWI_motion_corrected', maps=['original', 'coregistered', 'difference', 'mask', 'model_fit', 'deformation_field', 'parameters'])
dwi_registration.to_gif(output_directory=OUTPUT_DIR, base_file_name='DWI_motion_corrected', maps=['original', 'coregistered', 'difference', 'model_fit', 'deformation_field'])

Perform ADC and DTI calculation in the original and co-registered DWI images.

In [ ]:
# Calculate maps from the original diffusion sequence using ADC methods and save as niftis
adc_mapper = ADC(pixel_array, affine_dwi, bvals, ukrin_b=True)
adc_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='diffusion_original', maps='all')

# Calculate maps from the original diffusion sequence using DTI methods and save as niftis
dti_mapper = DTI(pixel_array, affine_dwi, bvals, bvecs, ukrin_b=True)
dti_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='diffusion_original', maps=['md', 'fa'])

# Calculate maps from the motion corrected diffusion sequence using ADC methods and save as niftis
adc_mapper_moco = ADC(dwi_moco_array, affine_dwi, bvals, ukrin_b=True)
adc_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='diffusion_moco', maps='all')

# Calculate maps from the motion corrected diffusion sequence using DTI methods and save as niftis
dti_mapper_moco = DTI(dwi_moco_array, affine_dwi, bvals, bvecs, ukrin_b=True)
dti_mapper.to_nifti(output_directory=OUTPUT_DIR, base_file_name='diffusion_moco', maps=['md', 'fa'])

Display the difference between the original DWI images and the co-registered images.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))

# Display the DWI original data
im = ax1.imshow(np.rot90(pixel_array[..., 4, 40]), clim=(0, 100000))
cb = fig.colorbar(im, ax=ax1)
cb.set_label('DWI Original')
ax1.axis('off')

# Display the DWI co-registered data
im2 = ax2.imshow(np.rot90(dwi_moco_array[..., 4, 40]), clim=(0, 100000))
cb = fig.colorbar(im2, ax=ax2)
cb.set_label('DWI Moco')
ax2.axis('off')

# Display the difference image between DWI original data and DWI co-registered data
im3 = ax3.imshow(np.rot90(dwi_moco_diff_array[..., 4, 40]), clim=(0, 100000))
cb = fig.colorbar(im3, ax=ax3)
cb.set_label('DWI Moco Difference')
ax3.axis('off')

plt.show()

Plot the ADC, MD and FA maps from the original images.

In [ ]:
# Display the central slice of each map
fig, ax = plt.subplots(2, 2, figsize=(16, 12))

# Display a central slice of the ADC map
im = ax[0, 0].imshow(np.rot90(adc_mapper.adc[:, :, adc_mapper.shape[2]//2]), cmap='inferno', clim=(0.001, 0.003))
cb = fig.colorbar(im, ax=ax[0, 0])
cb.set_label('ADC ($mm^2/s$)')
ax[0, 0].axis('off')

# Display a central slice of the MD map
im = ax[0, 1].imshow(np.rot90(dti_mapper.md[:, :, dti_mapper.shape[2]//2]), cmap='inferno', clim=(0.001, 0.003))
cb = fig.colorbar(im, ax=ax[0, 1])
cb.set_label('MD ($mm^2/s$)')
ax[0, 1].axis('off')

# Display a central slice of the FA map
im = ax[1, 0].imshow(np.rot90(dti_mapper.fa[:, :, dti_mapper.shape[2]//2]), cmap='viridis', clim=(0.1, 0.8))
cb = fig.colorbar(im, ax=ax[1, 0])
cb.set_label('FA')
ax[1, 0].axis('off')

# Display a central slice of the color FA map
im = ax[1, 1].imshow(np.rot90(dti_mapper.color_fa[:, :, dti_mapper.shape[2]//2, :]))
ax[1, 1].axis('off')

Plot the ADC, MD and FA maps from the co-registered images.

In [ ]:
# Display the central slice of each map
fig, ax = plt.subplots(2, 2, figsize=(16, 12))

# Display a central slice of the ADC map
im = ax[0, 0].imshow(np.rot90(adc_mapper_moco.adc[:, :, adc_mapper_moco.shape[2]//2]), cmap='inferno', clim=(0.001, 0.003))
cb = fig.colorbar(im, ax=ax[0, 0])
cb.set_label('ADC ($mm^2/s$)')
ax[0, 0].axis('off')

# Display a central slice of the MD map
im = ax[0, 1].imshow(np.rot90(dti_mapper_moco.md[:, :, dti_mapper_moco.shape[2]//2]), cmap='inferno', clim=(0.001, 0.003))
cb = fig.colorbar(im, ax=ax[0, 1])
cb.set_label('MD ($mm^2/s$)')
ax[0, 1].axis('off')

# Display a central slice of the FA map
im = ax[1, 0].imshow(np.rot90(dti_mapper_moco.fa[:, :, dti_mapper_moco.shape[2]//2]), cmap='viridis', clim=(0.1, 0.8))
cb = fig.colorbar(im, ax=ax[1, 0])
cb.set_label('FA')
ax[1, 0].axis('off')

# Display a central slice of the color FA map
im = ax[1, 1].imshow(np.rot90(dti_mapper_moco.color_fa[:, :, dti_mapper_moco.shape[2]//2, :]))
ax[1, 1].axis('off')